*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you will predict healthcare costs using a regression algorithm.

You are given a dataset that contains information about different people including their healthcare costs. Use the data to predict healthcare costs based on new data.

The first two cells of this notebook import libraries and the data.

Make sure to convert categorical data to numbers. Use 80% of the data as the `train_dataset` and 20% of the data as the `test_dataset`.

`pop` off the "expenses" column from these datasets to create new datasets called `train_labels` and `test_labels`. Use these labels when training your model.

Create a model and train it with the `train_dataset`. Run the final cell in this notebook to check your model. The final cell will use the unseen `test_dataset` to check how well the model generalizes.

To pass the challenge, `model.evaluate` must return a Mean Absolute Error of under 3500. This means it predicts health care costs correctly within $3500.

The final cell will also predict expenses using the `test_dataset` and graph the results.

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
dataset_path = keras.utils.get_file("insurance.csv", "https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv")
dataset = pd.read_csv(dataset_path)
dataset.tail()

In [ ]:
dataset.head()
dataset.describe() #analisis deskriptif pada fitur data numeric

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(dataset[['expenses', 'age', 'bmi', 'children']], alpha=0.3, diagonal='kde')
plt.figure(1)
plt.subplot(2,2,1)
dataset.groupby(['sex'])['expenses'].sum().plot.bar()
plt.subplot(2,2,2)
dataset.groupby(['smoker'])['expenses'].sum().plot.bar()
plt.subplot(2,2,3)
dataset.groupby(['region'])['expenses'].sum().plot.bar()
plt.figure(2)
plt.subplot(2,2,1)
dataset.groupby(['sex'])['bmi'].sum().plot.bar()
plt.subplot(2,2,2)
dataset.groupby(['smoker'])['bmi'].sum().plot.bar()
plt.subplot(2,2,3)
dataset.groupby(['region'])['bmi'].sum().plot.bar()

In [ ]:
dataset.isnull().sum()

In [ ]:
catColumns = ['sex', 'smoker', 'region']
dataset_dum = pd.get_dummies(dataset, columns = catColumns, drop_first=True)
dataset_dum.head()

In [ ]:
x = dataset_dum[['age', 'bmi', 'smoker_yes']]
y = dataset_dum['expenses']
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV
#membentuk estimator menggunakan regresi linear
linreg = LinearRegression()
#membentuk estimator menggunakan model reg ridge dengan CV =5
ridgereg = RidgeCV(alphas=(0.1,0.3,0.7,1.0),cv=5)
#membentuk estimator menggunakan model reg lasso dengan CV = 5
lassoreg = LassoCV(eps=0.001, cv = 5)
from sklearn.model_selection import cross_val_score

In [ ]:
linreg.fit(x,y) #membuat model regresi untuk x dan y
linreg.score(x,y) #skor regresi linear atau Nilai R square

In [ ]:
linreg.coef_

In [ ]:
linreg.intercept_

In [ ]:
def calc_insurance(age, bmi, smoking):
 y = ((age*linreg.coef_[0]) + (bmi*linreg.coef_[1]) + 
      (smoking*linreg.coef_[2]) - (linreg.intercept_))
 return y

In [ ]:
print(calc_insurance(36, 24, 0))


In [ ]:
dataset['sex'] = dataset['sex'].apply({'male':0,      'female':1}.get) 
dataset['smoker'] = dataset['smoker'].apply({'yes':1, 'no':0}.get)
dataset['region'] = dataset['region'].apply({'southwest':1, 'southeast':2, 'northwest':3, 'northeast':4}.get)

In [ ]:
import seaborn as sns
sns.jointplot(x=dataset['smoker'],y=dataset['expenses'])


In [ ]:
sns.jointplot(x=dataset['age'],y=dataset['expenses'])

In [ ]:
X = dataset[['age', 'sex', 'bmi', 'children','smoker','region']]
X.head()

In [ ]:
y = dataset['expenses']
y.head()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
len(X_test) # 402
len(X_train) # 936
len(dataset) # 1338

In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

model.fit(X_train,y_train)

In [ ]:
predictions = model.predict(X_test)

In [ ]:
predictions[0:5]

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test,predictions)
plt.xlabel('Y Test')
plt.ylabel('Predicted Y')

In [ ]:
data = {'age' : 40,
        'sex' : 1,
        'bmi' : 45.50,
        'children' : 4,
        'smoker' : 1,
        'region' : 3}
index = [1]
frank_df = pd.DataFrame(data,index)
frank_df

In [ ]:
prediction_frank = model.predict(frank_df)
print("Medical Insurance cost for Frank is : ",prediction_frank)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
